In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import os
import pickle
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

In [ ]:
# --- UPDATE THESE PATHS ---

# This is the path to the folder containing your 5,000 validation images
VAL_IMG_FOLDER = "/content/drive/My Drive/Evoastra - Team C Project/COCO2017_unzipped/val2017/"

# This is where your new feature file will be saved
OUTPUT_FILE_PATH = "/content/drive/My Drive/Evoastra - Team C Project/val_features.pkl"

# --- END OF PATHS ---

print(f"Loading images from: {VAL_IMG_FOLDER}")
print(f"Saving features to: {OUTPUT_FILE_PATH}")

In [ ]:
# Load the InceptionV3 model, pre-trained on ImageNet
# include_top=False removes the final classification layer
image_model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

# Create a new model that outputs the last convolutional layer
# This is our feature extractor
feature_extractor = Model(inputs=image_model.inputs, outputs=image_model.layers[-1].output)

print("InceptionV3 feature extractor model loaded.")

In [ ]:
def preprocess_image(image_path):
    # Load the image, resizing to 299x299 (required by InceptionV3)
    img = load_img(image_path, target_size=(299, 299))

    # Convert image to a 3D numpy array
    img_array = img_to_array(img)

    # Add a 4th dimension (batch size)
    img_array = tf.expand_dims(img_array, axis=0)

    # Preprocess the image for InceptionV3 (scales pixel values)
    img_array = inception_v3.preprocess_input(img_array)

    return img_array

print("Preprocessing function defined.")

In [ ]:
# Save the completed dictionary to your Google Drive
with open(OUTPUT_FILE_PATH, 'wb') as f:
    pickle.dump(features_dict, f)

print(f"Successfully saved all features to: {OUTPUT_FILE_PATH}")

In [ ]:
# --- 1. INSTALL AND IMPORT LIBRARIES ---
# We need to install the 'transformers' library from Hugging Face
!pip install transformers

import requests
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import matplotlib.pyplot as plt

print("--- Libraries installed and imported. ---")

# --- 2. LOAD THE PRE-TRAINED MODEL ---
# This will download the 'BLIP' model from Salesforce/Hugging Face
# It's a large model, so this might take 2-3 minutes.
print("Loading pre-trained model (Salesforce/blip-image-captioning-large)...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# If you have a GPU, move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("--- Model loaded successfully! ---")


# --- 3. LOAD THE TEST IMAGE ---
# We'll use the surfer image again to see the difference.
image_url = 'https://tensorflow.org/images/surf.jpg'
raw_image = Image.open(requests.get(image_url, stream=True).raw).convert('RGB')


# --- 4. GENERATE THE CAPTION ---
print("Generating caption...")

# Process the image (resize, normalize, etc.)
inputs = processor(raw_image, return_tensors="pt").to(device)

# Generate the caption (this tells the model to create text)
out = model.generate(**inputs)

# Decode the generated text
predicted_caption = processor.decode(out[0], skip_special_tokens=True)


# --- 5. SHOW THE RESULT ---
plt.imshow(raw_image)
plt.axis('off')
plt.show()

print(f"MODEL PREDICTION: {predicted_caption}")

In [ ]:
from google.colab import files
import io

print("--- Upload an image from your computer ---")
uploaded = files.upload()

# Get the filename of the uploaded file
for fn in uploaded.keys():
  print(f'User uploaded file "{fn}"')

  # Load the uploaded image
  raw_image = Image.open(io.BytesIO(uploaded[fn])).convert('RGB')

  # Generate the caption (using the model we already loaded)
  print("Generating caption...")
  inputs = processor(raw_image, return_tensors="pt").to(device)
  out = model.generate(**inputs)
  predicted_caption = processor.decode(out[0], skip_special_tokens=True)

  # Show the result
  plt.imshow(raw_image)
  plt.axis('off')
  plt.show()

  print(f"MODEL PREDICTION: {predicted_caption}")

In [ ]:
from google.colab import files
import io
import torch
from PIL import Image  # <-- [FIX] Import the Image library
import matplotlib.pyplot as plt

# --- [FIX] We must also re-define these variables ---
# The runtime forgot them when it restarted or timed out.
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# ---

print("--- Upload an image from your computer ---")
uploaded = files.upload()

# Get the filename of the uploaded file
for fn in uploaded.keys():
  print(f'User uploaded file "{fn}"')

  # Load the uploaded image
  raw_image = Image.open(io.BytesIO(uploaded[fn])).convert('RGB')

  # Generate the caption (using the model we already loaded)
  print("Generating caption...")
  inputs = processor(raw_image, return_tensors="pt").to(device)
  out = model.generate(**inputs)
  predicted_caption = processor.decode(out[0], skip_special_tokens=True)

  # Show the result
  plt.imshow(raw_image)
  plt.axis('off')
  plt.show()

  print(f"MODEL PREDICTION: {predicted_caption}")

In [ ]:
# --- 1. ALL IMPORTS ---
from google.colab import files
import io
import torch
from PIL import Image
import matplotlib.pyplot as plt
from transformers import BlipProcessor, BlipForConditionalGeneration

# --- 2. DEFINE MODEL AND PROCESSOR ---
print("Loading pre-trained model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model is ready.")

# --- 3. UPLOAD SCRIPT ---
print("\n--- Upload an image from your computer ---")
uploaded = files.upload()

# Get the filename of the uploaded file
for fn in uploaded.keys():
  print(f'User uploaded file "{fn}"')

  # Load the uploaded image
  raw_image = Image.open(io.BytesIO(uploaded[fn])).convert('RGB')

  # Generate the caption
  print("Generating caption...")
  inputs = processor(raw_image, return_tensors="pt").to(device)
  out = model.generate(**inputs)
  predicted_caption = processor.decode(out[0], skip_special_tokens=True)

  # Show the result
  plt.imshow(raw_image)
  plt.axis('off')
  plt.show()

  print(f"MODEL PREDICTION: {predicted_caption}")

In [ ]:
from google.colab import files
import io
import torch
from PIL import Image  # <-- [FIX] Import the Image library
import matplotlib.pyplot as plt

# --- [FIX] We must also re-define these variables ---
# The runtime forgot them when it restarted or timed out.
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# ---

print("--- Upload an image from your computer ---")
uploaded = files.upload()

# Get the filename of the uploaded file
for fn in uploaded.keys():
  print(f'User uploaded file "{fn}"')

  # Load the uploaded image
  raw_image = Image.open(io.BytesIO(uploaded[fn])).convert('RGB')

  # Generate the caption (using the model we already loaded)
  print("Generating caption...")
  inputs = processor(raw_image, return_tensors="pt").to(device)
  out = model.generate(**inputs)
  predicted_caption = processor.decode(out[0], skip_special_tokens=True)

  # Show the result
  plt.imshow(raw_image)
  plt.axis('off')
  plt.show()

  print(f"MODEL PREDICTION: {predicted_caption}")